In [ ]:
# Block 1
""" 
Change the following variable to the last revealed token_id
"""
last_revealed_token_id = 1489

In [ ]:
# Block 2
import pandas as pd
import requests
from utils import chain, config
from web3 import Web3

w3 = Web3(Web3.HTTPProvider(config.ENDPOINT))

PROVENANCE_HASH = "QmNQa5Vjf6hXfWZN1yFBd33VidHw1VFZdG7iyLMBVQSaA4"
FILE_NAME = "committed_HonestNFT_Vigilante_Metadata.csv"
CONTRACT_ADDRESS = "0xbD1d2Ea3127587f4ECFD271e1dADFc95320b8DeA"


"""
Download original metadata from IPFS and save to disk
"""
response = requests.get(f"{config.IPFS_GATEWAY}{PROVENANCE_HASH}/{FILE_NAME}")

if response.status_code == 200:
    with open(f"data/{FILE_NAME}", "wb") as f:
        f.write(response.content)

# Manually download the transactions from etherscan
# https://etherscan.io/exportData?type=address&a=0xbd1d2ea3127587f4ecfd271e1dadfc95320b8dea
# Earliest date is Mar-07-2022
# Rename it to data/transactions.csv

In [ ]:
# Block 3
""" 
Extract the transactions where the admin called "Request Random Words"
"""
transactions_df = pd.read_csv("data/transactions.csv", index_col=False)

transactions_df.sort_values(by=["UnixTimestamp"], inplace=True, ascending=True)

vrf_events_df = transactions_df.loc[transactions_df["Method"] == "Request Random Words"]
vrf_indexes = vrf_events_df.index.to_list()

In [ ]:
# Block 4
def get_token_id_from_tx(tx: str) -> int:
    """
    Given a transaction hash, return the minted token id
    """
    transaction_receipt = w3.eth.get_transaction_receipt(tx)
    token_id = Web3.toInt(transaction_receipt.get("logs")[-1].get("topics")[-1])
    return token_id


""" 
To determine the last minted token_id BEFORE the metadata was revealed, we look at the transactions before each VRF event.
If it was a mint transaction, we get the token_id and move on the the next vrf event.
"""
last_token_ids = [0]

for index in vrf_indexes:
    # Ensure the previous transaction is a mint event
    while "Mint" not in transactions_df.loc[index]["Method"]:
        index -= 1
    tx_hash = transactions_df.loc[index]["Txhash"]
    token_id = get_token_id_from_tx(tx_hash)
    last_token_ids.append(token_id)

In [ ]:
# Block 5
"""  
Get the seed list (random words) from the contract
"""

abi = chain.get_contract_abi(address=CONTRACT_ADDRESS)
abi, contract = chain.get_contract(address=CONTRACT_ADDRESS, abi=abi)
random_words_function = chain.get_contract_function(
    contract=contract, func_name="s_randomWords", abi=abi
)

seed_list = []

for index in range(len(vrf_indexes)):
    seed = random_words_function(index).call()
    seed_list.append(seed)

In [ ]:
# Block 6
""" 
This code block is almost identical to hermes_partial_shuffler.ipynb
"""
trait_sheet_unshuffled = pd.read_csv(
    "data/committed_HonestNFT_Vigilante_Metadata.csv", skiprows=1
)

full_trait_sheet = trait_sheet_unshuffled

for index in range(0, len(vrf_indexes)):
    seed = seed_list[index] % (2**32 - 1)
    last_revealed = last_token_ids[index]

    # get metadata for all unrevealed NFTs
    trait_sheet_to_shuffle = full_trait_sheet.tail(3777 - last_revealed)

    # shuffle
    trait_sheet_shuffled = trait_sheet_to_shuffle.sample(
        frac=1, random_state=seed
    ).reset_index(drop=True)
    # update token IDs for newly revealed NFTs
    trait_sheet_shuffled["TOKEN ID"] = trait_sheet_shuffled.index + 1 + last_revealed
    # append newly revealed NFTs to previously revealed NFTs
    new_trait_sheet = pd.concat(
        [full_trait_sheet[0:last_revealed], trait_sheet_shuffled]
    )
    full_trait_sheet = new_trait_sheet
    # output shuffled sheet

full_trait_sheet["TOKEN ID"] = full_trait_sheet["TOKEN ID"].astype(int)


full_trait_sheet.to_csv("data/shuffled_" + FILE_NAME, index=False)

In [ ]:
# Block 7
# Download the vigilante data with pulling.py
!python3 honestnft-shenanigans/metadata/pulling.py -contract $CONTRACT_ADDRESS -lower_id 1 -upper_id $last_revealed_token_id -max_supply $last_revealed_token_id

In [ ]:
# Block 8

vigilante_df = pd.read_csv(f"honestnft-shenanigans/data/raw_attributes/Vigilante.csv")
vigilante_df.sort_values(by=["TOKEN_ID"], inplace=True)

""" 
Some columns have a different name, so we must rename them for our comparison
"""
full_trait_sheet.rename(columns={"Masks": "Mask", "TOKEN ID": "TOKEN_ID"}, inplace=True)
full_trait_sheet.sort_values(by=["TOKEN_ID"], inplace=True)


"""
The full trait sheet contains both the feminine and masculine versions of the accessories.
But the vigilante data only contains the accessory that corresponds to the gender as 
specified in the full trait sheet. 
"""


def select_accessory_by_gender(df_series: pd.Series) -> str:
    gender = str(df_series["Gender"]).strip()
    if gender == "Feminine":
        return str(df_series["Accessory_F"])
    else:
        return str(df_series["Accessory_M"])


full_trait_sheet["Accessory"] = full_trait_sheet.apply(
    lambda row: select_accessory_by_gender(row), axis=1
)

""" 
Remove the columns that don't exist in the respective DataFrames
"""
full_trait_sheet = full_trait_sheet.drop(
    columns=["Accessory_F", "Accessory_M", "Image_Number"]
)
vigilante_df = vigilante_df.drop(columns=["TOKEN_NAME"])

""" 
Sort the columns for easier comparison
"""
full_trait_sheet = full_trait_sheet.reindex(sorted(full_trait_sheet.columns), axis=1)
vigilante_df = vigilante_df.reindex(sorted(vigilante_df.columns), axis=1)

""" 
Remove all the unrevealed rows from the full trait sheet
"""
full_trait_sheet = full_trait_sheet.loc[
    full_trait_sheet["TOKEN_ID"] <= last_revealed_token_id
]

""" 
Make sure both DataFrames use TOKEN_ID as the index
"""
full_trait_sheet.set_index("TOKEN_ID", inplace=True)
vigilante_df.set_index("TOKEN_ID", inplace=True)


"""
The last step is comparing the two DataFrames.
"""
if full_trait_sheet.equals(vigilante_df):
    print("The trait sheets are identical.")
else:
    print("We got rugged!\nThese rows don't match:")
    print(full_trait_sheet.compare(vigilante_df))